In [3]:
import math
class Vector3D:
    """
    3D Vector class

    A 3D vector is viewed as a linear combination of
    basis vectors: x*i-hat +  y*j-hat + z*k-hat

    Attributes:
        data: [x,y,z]

    Example:
        >>> v = Vector3D([1,1,2])
    """
    def __init__(self, components):
        if (len(components)) != 3:
            raise ValueError("Must have exactly 3 components")
        self.components = components
        
    def __repr__(self):
        return f"Vector({self.components})"
        
    def __add__(self, other):
        """ Add two vectors component-wise"""
        if len(self.components) != len(other.components):
            raise ValueError("Vectors must be the same dimension")
        result = [a+b for a,b in zip(self.components, other.components)]
        return Vector3D(result)
        
    def __mul__(self, scalar):
        """Multiply vector by a scalar"""
        result = [scalar * component for component in self.components]
        return Vector3D(result)
        
    def magnitude(self):
        """
        Compute the length of the vector.

        The magnitude of the vector determines the
        distance from the  origin.

        Returns:
            float: magnitude
        Formula:
            Pythagorean Theorem: square root(a**2 + b**2 + c**2)
        Example:
            >>> v = Vector3D([3, 4, 0])
            >>> v.magnitude()
            5.0
        """
        # use pythagorean theorem
        sum_of_squares = sum(c**2 for c in self.components)
        return sum_of_squares ** .5
        
    def dot(self, other):
        """
        Computes the dot product with another vector.

        The dot product is a measure of how aligned two vectors are.
        If dot = 0, the vectors are perpendicular, and the angle is 90 degrees. If > 0, the 
        vectors point in similar direction and the angle trends towards 0 degrees. If < 0, 
        the vectors point in  opposite directions and the angle trends towards 180 degrees.

        Args:
            other: Vector3D to compute dot product with
        
        Returns:
            Float Dot Product
        
        Formula:
            |a| * |b| * cos(θ), a1*b1 + ... + ax*bx

        Example:
            >>> v1 = Vector3D([1, 0, 0])
            >>> v2 = Vector3D([0, 1, 0])
            >>> v1.dot(v2)
            0  # Perpendicular vectors  
        """
        if len(self.components) != len(other.components):
            raise ValueError("Vectors must be the same dimension")  
        return sum(a*b for a,b in zip(self.components, other.components))
        
    def normalize(self):
        """
        Maintains vector direction while reducing magnitude to one.

        Normalizing a vector transforms it into a unit vector. A unit
        vector represents pure direction and can be scaled into any
        magnitude.

        Formula:
            Each component / magnitude

        Use Cases:
            - Representing pure direction
            - Normalizing Data for ML algorithms
            - Lighting Calculation for Graphics/Game Engines
        
        Returns:
            Vector3D unit vector (magnitude = 1) in the same direction

        Example:
            >>> v = Vector3D([3, 4, 0])
            >>> v.normalize()
            Vector3D([0.6, 0.8, 0.0])  # magnitude = 1
        """
        return Vector3D([component / self.magnitude() for component in self.components])

    def cross(self, other):
        """
        Returns a vector normal to self and other.

        The cross product produces a new vector that is normal (perpendicular)
        to the plane formed by the two input vectors. The direction is determined
        by the right-hand rule - Index finger - self, Middle Finger - other, thumb - cross product This
        rule is anti-commutative - a*b = -(b*a). The magnitude of the cross product
        represents the area of the parallelogram formed by the two input vectors.

        Formula:
            x = a2*b3 -a3*b2
            y = a3*b1 - a1*b3
            z = a1*b2 - a2*b1

        Args:
            other: Vector3D

        Returns:
            Vector3D

        Example:
            >>> v1 = Vector3D([1,0,0])
            >>> v2 = Vector3D([0,1,0])
            v1.cross(v2) # [0,0,1]
            v2.cross(v1) # [0,0,-1]
        """
        x = (self.components[1]*other.components[2]) - (self.components[2]*other.components[1])
        y = (self.components[2]*other.components[0]) - (self.components[0]*other.components[2])
        z = (self.components[0]*other.components[1]) - (self.components[1]*other.components[0])

        return Vector3D([x,y,z])


# Cross Product Testing
v1,v2 = Vector3D([2,0,0]), Vector3D([0,3,0])
cross = v1.cross(v2)
print(cross)
if  ((cross.dot(v1) == 0) and (cross.dot(v2) == 0)):
    print("perpendicular!")
else:
    print("not perpendicular!")

v3, v4 = Vector3D([1,2,0]), Vector3D([0,1,3])
cross2 = v3.cross(v4)
print(cross2)
if  ((cross2.dot(v3) == 0) and (cross2.dot(v4) == 0)):
    print("perpendicular!")
else:
    print("not perpendicular!")

#parallel
v5,v6 = Vector3D([1,2,3]), Vector3D([2,4,6])
cross3 = v5.cross(v6)
print(cross3)
if  ((cross3.dot(v5) == 0) and (cross3.dot(v6) == 0)):
    print("perpendicular!")
else:
    print("not perpendicular!")

        
        

Vector([0, 0, 6])
perpendicular!
Vector([6, -3, 1])
perpendicular!
Vector([0, 0, 0])
perpendicular!


In [52]:
import math
from typing import List, Union
from Vector3D import Vector3D

class Matrix3D:
    """
    Matrix3D Class representing linear transformations.

    A matrix is viewed geometrically as a transformation that maps vectors 
    to new vectors. Each column represents where the basis vectors land
    after the transformation. Matrices should be 3D or higher and square.

    Attributes:
        data: 2D list of numbers representing the matrix
        rows: number of rows in the matrix
        cols: number of columns in the matrix
        
    Example:
        >>> M = Matrix3D([[2,0,0], [0,3,0], [0,0,5]]) # Scaling transformation
        >>> v = Vector3D([1,2,3])
        >>> result = M.multiply_vector(v) # should give Vector3D([2,6,15])
    """
    def __init__(self, data: List[List[float]]):
        """
        Initialize matrix from 2D list.

        Args:
            data: 2D list where each inner list is a row
                ex: [[1,2], [3,4]] represents a 2x2 matrix.
        """
        self.data = data
        self.rows = len(data) #counts number of inner lists (rows)
        self.cols = len(data[0]) if data else 0 # counts how many elements are in first row (columns). checks for empty matrix
        if (self.rows < 3 or self.cols < 3):
            raise ValueError("Matrix must at least be 3x3.")
        if (self.rows != self.cols):
            raise ValueError("Matrix not square")

    def __repr__(self) -> str:
        """
        Return string representation for debugging.

        Returns:
            String like "Matrix3D([[1,2], [3,4])"
        """
        return f"Matrix3D({self.data})"
    
    def __str__(self) -> str:
        """

        Pretty print the matrix in readable format.

        Returns:
         Multi-line string with formatted matrix
         Example:
             [1.00, 2.00, 3.00]
             [4.00, 5.00, 6.00]
        """
        rowstrings = []
        for row in self.data:
            rstring = ", ".join(f"{num:.2f}" for num in row)
            rowstrings.append(rstring)

        return "\n".join(rowstrings)

    def get_column(self, col_index: int) -> Vector3D:
        """
        Extract a column as a vector.

        Geometric: The i-th column shows where the i-th basis vector lands
        after this transformation.

        Args:
            col_index: Which column to extract (0-indexed)
        Returns:
            Vector3D containing the column values
        """
        column = [row[col_index] for row in self.data]
        return Vector3D(column)

    def multiply_vector(self, vector: Vector3D) -> Vector3D:
        """
        Apply this transformation to a vector (matrix-vector multiplication

        Geometric: Where does this vector land after the transformation 
        represented by this matrix?

        Math: Each component of result is a dot product of a matrix row
        with the input vector.

        Args:
            vector: Vector3D
        
        Returns:
            Vector3D
        
        Raises: ValueError: If matrix columns don't match vector dimension

        Example:
            >>>M = Matrix3D([[2,0, 0], [0,3, 0], [0,0,4)] #scale x by 2, y by 3, z by 4)
            >>>v = Vector3D([1,1,1]) # returns Vector3D([2,3,4])
        """
        if (len(vector.components) != self.cols):
            raise ValueError("Dimensions don't match columns")
        result = [vector.dot(Vector3D(row)) for row in self.data]
        return Vector3D(result)

    def multiply_matrix(self, other: 'Matrix3D') -> 'Matrix3D':
        """ 
        Applies a transformation to the matrix.

        Geometric interpretation: The returned matrix represents a composed transformation
        that first applies other, then applies self.

        Mathematical: Each component of the Matrix is a dot product of the original matrix's 
        rows and the transformation matrix's columns.

        Order: Order matters here. self * other != other * self.

        Raises: ValueError: If Matrix Inner Dimensions don't match: mxn nxm works, nxm nxm does not.

        """
        if (self.cols != other.rows):
            raise ValueError("Inner dimensions don't match")
        product = []
        for i in range(other.cols):
            product.append(self.multiply_vector(other.get_column(i)).components)
        toReturn = []
        for row_index in range(len(product[0])):
            new_row = []
            for column in product:
                new_row.append(column[row_index])
            toReturn.append(new_row)
        return Matrix3D(toReturn)
                
            
        

    @staticmethod
    def rotation(angle_degrees: float, axis: str) -> 'Matrix3D':
        """
        Create a 3D rotation matrix around the specified axis.

        Args:
            angle_degrees: Rotation angle in degrees
            axis: which axis to rotate around("x", "y", or "z")
        
        Returns:
            3x3 rotation matrix
        """
        radians = math.radians(angle_degrees)
        cos = math.cos(radians)
        sin = math.sin(radians)
        negSin = -math.sin(radians)

        match axis:
            case "x":
                r1 = [1, 0, 0]
                r2 = [0, cos, negSin]
                r3 = [0, sin, cos]
            case "y":
                r1 = [cos, 0, sin] 
                r2 = [0, 1, 0]
                r3 = [negSin, 0, cos]
            case "z":
                r1 = [cos, negSin, 0]
                r2 = [sin, cos, 0]
                r3 = [0, 0, 1]
            case _:
                raise ValueError("Axis mismatch: Only x, y, or z is supported")
        return Matrix3D([r1,r2,r3])
                
        

        return Matrix([r1,r2])

    @staticmethod
    def scaling(*scales) -> 'Matrix3D':
        """
        Create a nD scaling matrix.

        Scales n coords by sn

        Args:
            *scales variable number of scale factors (one per dimension)

        Returns:
           nxn scaling matrix
        """
        n = len(scales)
        rows = []

        # [[scales[i] if i == j else 0 for j in range(n)]for i in range(n)] - nested list comprehension
        
        for i in range(n):
            row_n = []
            for j in range(n):
                if i == j: 
                    row_n.append(scales[i])
                else:
                    row_n.append(0)
            rows.append(row_n)
        return Matrix3D(rows)

    @staticmethod
    def identity(size=3) -> 'Matrix3D':
        """
        Create an identy matrix of given size

        Arg:
            Size of matrix
            
        Return:
            Identity Matrix
        """
        identity = [[1 if i == j else 0 for j in range(size)] for i in range(size)]

        return Matrix3D(identity)
    

   

m = Matrix3D([[1,0,0], [0,2,0],[0,0,3]])
print(m)
col1 = m.get_column(0)
print(col1)
v = Vector3D([1,2,3])
m2 = m.multiply_vector(v)
print(m2)
m3 = m.multiply_matrix(m)
print(m3)
# Test rotation around z-axis by 90 degrees
# Should rotate (1, 0, 0) to approximately (0, 1, 0)
Rz = Matrix3D.rotation(90, "z")
v = Vector3D([1, 0, 0])
result = Rz.multiply_vector(v)
print(result)  # Should be approximately [0, 1, 0
Rx = Matrix3D.rotation(90, "x")
result = Rx.multiply_vector(v)
print(result)
Ry = Matrix3D.rotation(90, "y")
result = Ry.multiply_vector(v)
print(result)

# 3D scaling (what you originally wanted)
S3 = Matrix3D.scaling(2, 3, 4)
print(S3)
print()

# 4D scaling (just because you can!)
S4 = Matrix3D.scaling(2, 3, 4, 5)
print(S4)
print()

# Test it with a vector
v = Vector3D([1, 1, 1])
result = S3.multiply_vector(v)
print(result)  # Should be [2, 3, 4]





1.00, 0.00, 0.00
0.00, 2.00, 0.00
0.00, 0.00, 3.00
Vector([1, 0, 0])
Vector([1, 4, 9])
1.00, 0.00, 0.00
0.00, 4.00, 0.00
0.00, 0.00, 9.00
Vector([6.123233995736766e-17, 1.0, 0])
Vector([1, 0.0, 0.0])
Vector([6.123233995736766e-17, 0, -1.0])
2.00, 0.00, 0.00
0.00, 3.00, 0.00
0.00, 0.00, 4.00

2.00, 0.00, 0.00, 0.00
0.00, 3.00, 0.00, 0.00
0.00, 0.00, 4.00, 0.00
0.00, 0.00, 0.00, 5.00

Vector([2, 3, 4])


In [54]:

from matrix3D import Matrix3D
from Vector3D import Vector3D
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib.animation import FuncAnimation


def lerp_matrix(start: Matrix3D, end: Matrix3D, t: float):
    """
    Linearly interpolate between two matrices.

    Args:
        start: Matrix3D
        end: Matrix3D
        t: float
    
    Return:
        interpol: Matrix3D
    """
    if (start.rows != end.rows or start.cols != end.cols): raise ValueError("Dimensions don't match")
    if (t < 0 or t > 1): raise ValueError("T must be a value between 0 and 1")

    interpol = [[(start.data[i][j] + (end.data[i][j]-start.data[i][j])*t) 
                 for j in range(start.cols)] for i in range(start.rows)]
    
    return Matrix3D(interpol)



# Create 3D Plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#Define Basis Vectors
i_hat = Vector3D([1,0,0])
j_hat = Vector3D([0,1,0])
k_hat = Vector3D([0,0,1])
t_mat = Matrix3D.rotation(45, "z")

i_transformed = t_mat.multiply_vector(i_hat)
j_transformed = t_mat.multiply_vector(j_hat)
k_transformed = t_mat.multiply_vector(k_hat)
scale = 3

quiver_i = ax.quiver(0,0,0,i_transformed.components[0]*scale,i_transformed.components[1]*scale,i_transformed.components[2]*scale,
          color='orange', arrow_length_ratio=0.1, linewidth=3, label='i-hat')
quiver_j = ax.quiver(0,0,0,j_transformed.components[0]*scale, j_transformed.components[1]*scale, j_transformed.components[2]*scale,
          color='yellow', arrow_length_ratio=0.1, linewidth=3, label='j-hat')
quiver_k = ax.quiver(0,0,0,k_transformed.components[0]*scale, k_transformed.components[1]*scale, k_transformed.components[2]*scale,
          color='cyan', arrow_length_ratio=0.1, linewidth=3, label='k-hat')
quivers = {'i' : quiver_i, 'j' : quiver_j, 'k' : quiver_k}

x_points = np.linspace(-2,2,9)
x_vectors = [Vector3D([x,0,0]) for x in x_points]
transformed_x = [t_mat.multiply_vector(x) for x in x_vectors]
x_coords = [vector.components[0] for vector in transformed_x]
y_coords = [vector.components[1] for vector in transformed_x]
z_coords = [vector.components[2] for vector in transformed_x]

 # Y-axis line
y_points = np.linspace(-2, 2, 9)
y_vectors = [Vector3D([0, y, 0]) for y in y_points]
transformed_y = [t_mat.multiply_vector(y) for y in y_vectors]
"""
y_coords_t = [v.components[0] for v in transformed_y]
y_coords_t_y = [v.components[1] for v in transformed_y]
y_coords_t_z = [v.components[2] for v in transformed_y]
ax.plot(y_coords_t, y_coords_t_y, y_coords_t_z, color='magenta', label='y-axis line')
"""
# Z-axis line
z_points = np.linspace(-2, 2, 9)
z_vectors = [Vector3D([0, 0, z]) for z in z_points]
transformed_z = [t_mat.multiply_vector(z) for z in z_vectors]
"""
z_coords_t = [v.components[0] for v in transformed_z]
z_coords_t_y = [v.components[1] for v in transformed_z]
z_coords_t_z = [v.components[2] for v in transformed_z]
ax.plot(z_coords_t, z_coords_t_y, z_coords_t_z, color='lime', label='z-axis line')

ax.plot(x_coords, y_coords, z_coords, color='magenta')
"""

"""
#plot as arrows from origin
#ax.quiver - (start_x, start_y, start_z, dir_x dir_y dir_z
ax.quiver(0,0,0,1,0,0, color='r', arrow_length_ratio=0.1, label='i-hat')
ax.quiver(0,0,0,0,1,0, color='g', arrow_length_ratio=0.1, label='j-hat')
ax.quiver(0,0,0,0,0,1, color='b', arrow_length_ratio=0.1, label='k-hat')
"""
x_lines = []
y_lines = []
z_lines = []
grid_positions = np.linspace(-2,2,9)
for y in grid_positions:
    for z in grid_positions:
        x_vectors = [Vector3D([x,y,z]) for x in x_points]
        transformed_x = [t_mat.multiply_vector(v) for v in x_vectors]
        grid_x = [x.components[0] for x in transformed_x]
        grid_y = [x.components[1] for x in transformed_x]
        grid_z = [x.components[2] for x in transformed_x]

        line = ax.plot(grid_x, grid_y, grid_z,color='green', alpha=.3)
        x_lines.append(line)
        

for x in grid_positions:
    for z in grid_positions:
        y_vectors = [Vector3D([x,y,z]) for y in y_points]
        transformed_y = [t_mat.multiply_vector(v) for v in y_vectors]
        grid_x = [y.components[0] for y in transformed_y]
        grid_y = [y.components[1] for y in transformed_y]
        grid_z = [y.components[2] for y in transformed_y]

        line = ax.plot(grid_x, grid_y, grid_z, color='blue', alpha=.3)
        y_lines.append(line)
        

for x in grid_positions:
    for y in grid_positions:
        z_vectors = [Vector3D([x,y,z]) for z in z_points]
        transformed_z = [t_mat.multiply_vector(v) for v in z_vectors]
        grid_x = [z.components[0] for z in transformed_z]
        grid_y = [z.components[1] for z in transformed_z]
        grid_z = [z.components[2] for z in transformed_z]

        line = ax.plot(grid_x, grid_y, grid_z, color='red', alpha=.3)
        z_lines.append(line)

# set axis limits and labels
ax.set_xlim([-2,2])
ax.set_ylim([-2,2])
ax.set_zlim([-2,2])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

def update(frame):
    t = frame/max_frames
    quivers['i'].remove()
    quivers['j'].remove()
    quivers['k'].remove()

    start = Matrix3D.identity()
    current_matrix = lerp_matrix(start,t_mat, t)

    idx = 0
    idy = 0
    idz = 0
    #update x parallel lines
    for y in grid_positions:
        for z in grid_positions:
            x_vectors = [Vector3D([x,y,z]) for x in x_points]
            transformed = [current_matrix.multiply_vector(v) for v in x_vectors]
            xs = [v.components[0] for v in transformed]
            ys = [v.components[1] for v in transformed]
            zs = [v.components[2] for v in transformed]

            x_lines[idx][0].set_data_3d(xs,ys,zs)
            idx+=1
    #update y parallel lines
    for x in grid_positions:
        for z in grid_positions:
            y_vectors = [Vector3D([x,y,z]) for y in y_points]
            transformed = [current_matrix.multiply_vector(v) for v in y_vectors]
            xs = [v.components[0] for v in transformed]
            ys = [v.components[1] for v in transformed]
            zs = [v.components[2] for v in transformed]

            y_lines[idy][0].set_data_3d(xs,ys,zs)
            idy+=1
    #update z parallel lines
    for x in grid_positions:
        for y in grid_positions:
            z_vectors = [Vector3D([x,y,z]) for z in z_points]
            transformed = [current_matrix.multiply_vector(v) for v in z_vectors]
            xs = [v.components[0] for v in transformed]
            ys = [v.components[1] for v in transformed]
            zs = [v.components[2] for v in transformed]

            z_lines[idz][0].set_data_3d(xs,ys,zs)
            idz+=1
    #update basis vectors
    i_transformed = current_matrix.multiply_vector(i_hat)
    j_transformed = current_matrix.multiply_vector(j_hat)
    k_transformed = current_matrix.multiply_vector(k_hat)

    quivers['i'] = ax.quiver(0,0,0,i_transformed.components[0]*scale,i_transformed.components[1]*scale,i_transformed.components[2]*scale,
          color='orange', arrow_length_ratio=0.1, linewidth=3, label='i-hat')
    quivers['j'] = ax.quiver(0,0,0,j_transformed.components[0]*scale, j_transformed.components[1]*scale, j_transformed.components[2]*scale,
              color='yellow', arrow_length_ratio=0.1, linewidth=3, label='j-hat')
    quivers['k'] = ax.quiver(0,0,0,k_transformed.components[0]*scale, k_transformed.components[1]*scale, k_transformed.components[2]*scale,
              color='cyan', arrow_length_ratio=0.1, linewidth=3, label='k-hat')

    

    return []
max_frames = 60
anim = FuncAnimation(fig, update, frames=max_frames, interval=50, blit=False)
anim.save('transformation_animation.gif', writer='pillow', fps=20)
print("Animation saved!")

plt.show()
start = Matrix3D([[1,1,1],[1,1,1],[1,1,1]])
end = Matrix3D([[2,2,2],[2,2,2],[2,2,2]])
inter = lerp_matrix(start,end,.5)
print(inter)

<IPython.core.display.Javascript object>

IndexError: list index out of range